# 自动求梯度


每个Tensor都有一个.grad_fn属性，该属性即创建该Tensor的Function, 就是说该Tensor是不是通过某些运算得到的

In [3]:
import torch
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [4]:
y = x + 2
print(y)
print(y.grad_fn)
# 注意x是直接创建的，所以它没有grad_fn, 而y是通过一个加法操作创建的，所以它有一个为<AddBackward>的grad_fn。

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [5]:
print(x.is_leaf, y.is_leaf)

True False


In [6]:
z = y * y * 3
out = z.mean()
print(z, out)


tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


通过.requires_grad_()来用in-place的方式改变requires_grad属性

## 梯度

In [ ]:
out.backward() # 等价于 out.backward(torch.tensor(1.))

In [10]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


注意：grad在反向传播过程中是累加的(accumulated)，这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以一般在反向传播之前需把梯度清零。

In [11]:
out2 = x.sum()
out2.backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])


不允许张量对张量求导，只允许标量对张量求导，求导结果是和自变量同形的张量

In [18]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


现在 z 不是一个标量，所以在调用backward时需要传入一个和z同形的权重向量进行加权求和得到一个标量

In [19]:
v = torch.tensor([[1.0, 1.5], [1.0, 1.0]], dtype=torch.float)
z.backward(v)
print(x.grad)
# x.grad是和x同形的张量

tensor([2., 3., 2., 2.])
